# Öğrenme Aktarımı (Transfer Learning)

In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
from tqdm import tqdm # Eğitim sürecini takip etmek için kullandığımız ilerleme çubuğu
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix , classification_report

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

### 1)Veri Yükleme (Data Load) ve Veri Arttırımı (Data Agumentation)

In [6]:
transform_train = transforms.Compose([
    transforms.Resize((224,224)) , # Mobilnet input size
    transforms.RandomHorizontalFlip(), # Görüntüleri yatay çevirerek veri arttırımı
    transforms.RandomRotation(10) , # # Görüntüleri 10° derece kadar döndürür
    transforms.ColorJitter(brightness = 0.2 , contrast = 0.2 , saturation = 0.2 , hue = 0.1) , # Renk varyasyonları
    transforms.ToTensor() , # Görüntüleri tensore çevir
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))  # Piksel değerlerini normalize etme işlemi
 ])

In [8]:
transform_test = transforms.Compose([
    transforms.Resize((224,224)) , 
    transforms.ToTensor() ,
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

**Oxford Flowers 102 Veri seti yükleme**

In [12]:
train_dataset = datasets.Flowers102(root = "./Flowers102_data" , split = "val" , transform = transform_train , download = False)
test_dataset = datasets.Flowers102(root = "./Flowers102_data" , split = "val" , transform = transform_test , download = False)

In [13]:
train_loader = DataLoader(train_dataset , batch_size = 32 , shuffle = True)
test_loader = DataLoader(test_dataset ,batch_size = 32 , shuffle = False)

### 2)Transfer Learning

In [16]:
# Mobilnet v2 yükleme
model = models.mobilenet_v2(pretrained = True) # Pretrained = True : Önceden eğitilmiş ağırlıklar

# Sınıflandırıcı katmanı ekleme
num_ftrs = model.classifier[1].in_features # Mevcut sınıflandırıcının giriş özzelliklerini alma işlemi

# Son katmanı oxford flower 102 için değiştir
model = model.to(device)

# Kayıp fonk. ve optimizer tanımlama
criterion = nn.CrossEntropyLoss() #Çok sınıflı sınıflandırma için çapraz entropi kullanımı

optimizer = optim.Adam(model.classifier[1].parameters() , lr = 0.001) # Sadece son katmanı optimize etme işlemi

scheduler = optim.lr_scheduler.StepLR(optimize , step_size = 5 , gamma = 0.1) # Step lr

C:\Users\emirh\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\emirh\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


**Model Eğitimi**

In [17]:
epochs = 5
for epoch in tqdm(range(epochs)):
    model.train() # Modeli eğitim moduna alma işlemi
    running_loss = 0.0 # Toplam kayıp değeri
    
    for images , labels in train_loader:
        images , labels = images.to(device) , labels.to(device)
        optimizer.zero_grad() # Önceki gradyanları sıfırla
        outputs = model(images) # Model ile tahmin işlemi
        loss = criterion(outputs , labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    scheduler.step()
    print(f"Epoch : {epoch+1} - Loss {running_loss/len(train_loader):.4f}")

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]C:\Users\emirh\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:182: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
 20%|████████████████▌                                                                  | 1/5 [01:52<07:29, 112.32s/it]

Epoch : 1 - Loss 7.9577


 40%|█████████████████████████████████▌                                                  | 2/5 [03:01<04:20, 86.86s/it]

Epoch : 2 - Loss 5.6387


 60%|██████████████████████████████████████████████████▍                                 | 3/5 [04:20<02:46, 83.42s/it]

Epoch : 3 - Loss 4.2709


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [05:27<01:16, 76.82s/it]

Epoch : 4 - Loss 3.1007


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [06:33<00:00, 78.67s/it]

Epoch : 5 - Loss 2.4897


**Modeli Kaydetme**

In [18]:
torch.save(model.state_dict(),"mobilenet_flowers102.pth")

In [19]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images , labels in tqdm(test_loader):
        images , labels = images.to(device) , labels.to(device)
        outputs = model(images)
        _ , predicted = torch.max(outputs,1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

cm = confusion_matrix(all_labels , all_preds)
cm

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:26<00:00,  1.22it/s]


array([[2, 0, 0, ..., 0, 0, 0],
       [0, 7, 0, ..., 0, 0, 0],
       [0, 0, 3, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 7, 0, 0],
       [0, 0, 0, ..., 0, 5, 0],
       [0, 0, 0, ..., 0, 0, 4]], dtype=int64)

In [21]:
print(classification_report(all_labels,all_preds))

              precision    recall  f1-score   support

           0       0.17      0.20      0.18        10
           1       0.58      0.70      0.64        10
           2       0.50      0.30      0.38        10
           3       0.25      0.20      0.22        10
           4       0.14      0.20      0.17        10
           5       0.82      0.90      0.86        10
           6       0.29      0.20      0.24        10
           7       0.64      0.70      0.67        10
           8       1.00      0.40      0.57        10
           9       0.69      0.90      0.78        10
          10       0.33      0.20      0.25        10
          11       0.82      0.90      0.86        10
          12       0.89      0.80      0.84        10
          13       1.00      0.90      0.95        10
          14       0.44      0.40      0.42        10
          15       0.50      0.40      0.44        10
          16       0.53      1.00      0.69        10
          17       0.60    

C:\Users\emirh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\emirh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\emirh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
